In [ ]:
from google.colab import files
uploaded = files.upload()

Saving company_data.csv to company_data.csv
Saving influencers_data.csv to influencers_data.csv


In [ ]:
import io
import pandas as pd
dfc = pd.read_csv(io.BytesIO(uploaded['company_data.csv']))
dfi = pd.read_csv(io.BytesIO(uploaded['influencers_data.csv']))

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (18) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
print(dfc.shape)
print(dfi.shape)

(11654, 19)
(34012, 19)


In [ ]:
dfc[0:9]

In [ ]:
dfi[0:9]

In [ ]:
print(dfc.isnull().values.sum(axis=0))
print(dfi.isnull().values.sum(axis=0))

[    0     0    30    30    30     0     0     0   283     0   640     0
     0     0     0     0     0 11654 11607]
[    0     0     0  2272    42  8299     0     1  2016     0  7233     0
     0     0     0     0     0 34012 33926]


In [ ]:
#removes unnecessary columns from dataset
dfc=dfc.drop(["location", "media_type", "views", "votes"], axis=1)
dfc[0:9]

,Unnamed: 0,name,headline,followers,connections,about,time_spent,content,content_links,media_urls,num_hashtags,hashtag_followers,hashtags,reactions,comments
0,0,Dell Technologies,Information Technology & Services,"3,597,845 followers","194,140",Dell Technologies is a unique family of busine...,1 week ago,#ICYMI check out the keynote replay from the ...,[['https://www.linkedin.com/feed/hashtag/?keyw...,['blob:https://www.linkedin.com/4680e2eb-c683-...,2,0,[['https://www.linkedin.com/feed/hashtag/?keyw...,1217,577
1,1,Dell Technologies,Information Technology & Services,"3,597,845 followers","194,140",Dell Technologies is a unique family of busine...,20 hours ago,“We need to move faster to address the digital...,[['https://www.linkedin.com/in/ACoAAABpTIwBhTx...,['https://media-exp1.licdn.com/dms/image/C4D22...,1,0,[['https://www.linkedin.com/feed/hashtag/?keyw...,76,2
2,2,Dell Technologies,Information Technology & Services,"3,597,845 followers","194,140",Dell Technologies is a unique family of busine...,20 hours ago,"Meet our 2021 APJ Sales Graduates! Last week,...",[['https://www.linkedin.com/feed/hashtag/?keyw...,['https://media-exp1.licdn.com/dms/image/C4D22...,2,0,[['https://www.linkedin.com/feed/hashtag/?keyw...,35,0
3,3,Dell Technologies,Information Technology & Services,"3,597,845 followers","194,140",Dell Technologies is a unique family of busine...,22 hours ago,"From our family to yours, we wish you four sea...",[],['https://dms.licdn.com/playlist/C5605AQGCmtEZ...,0,0,[],195,2
4,4,Dell Technologies,Information Technology & Services,"3,597,845 followers","194,140",Dell Technologies is a unique family of busine...,2 days ago,#5G will revolutionize food production — soil...,[['https://www.linkedin.com/feed/hashtag/?keyw...,[],3,0,[['https://www.linkedin.com/feed/hashtag/?keyw...,250,6
5,5,Dell Technologies,Information Technology & Services,"3,597,845 followers","194,140",Dell Technologies is a unique family of busine...,2 days ago,Rapid change and unexpected circumstances are ...,"[['https://dell.to/3a1bM7x', 'https://dell.to/...",['https://media-exp1.licdn.com/dms/image/C4D22...,0,0,[],144,0
6,6,Dell Technologies,Information Technology & Services,"3,597,845 followers","194,140",Dell Technologies is a unique family of busine...,4 days ago,According to Gartner's 2020 CEO survey most bu...,"[['https://dell.to/3qeNkps', 'https://dell.to/...",['https://media-exp1.licdn.com/dms/image/C4D22...,1,0,[['https://www.linkedin.com/feed/hashtag/?keyw...,128,0
7,7,Dell Technologies,Information Technology & Services,"3,597,845 followers","194,140",Dell Technologies is a unique family of busine...,5 days ago,A true All-in-One experience to empower your p...,[['https://www.linkedin.com/feed/hashtag/?keyw...,['https://media-exp1.licdn.com/dms/image/C4D22...,1,0,[['https://www.linkedin.com/feed/hashtag/?keyw...,410,5
8,8,Dell Technologies,Information Technology & Services,"3,597,845 followers","194,140",Dell Technologies is a unique family of busine...,1 day ago,Imagine the future of work as better connected...,[['https://www.linkedin.com/feed/hashtag/?keyw...,['https://dms.licdn.com/playlist/C4D05AQHjzm4O...,1,0,[['https://www.linkedin.com/feed/hashtag/?keyw...,167,3


In [ ]:
#removes unnecessary columns from dataset
dfi=dfi.drop(["location", "media_type", "views", "votes"], axis=1)
dfi[0:9]

,Unnamed: 0,name,headline,followers,connections,about,time_spent,content,content_links,media_url,num_hashtags,hashtag_followers,hashtags,reactions,comments
0,0,Nicholas Wyman,CEO IWSI Group,6484.0,500+,Nicholas Wyman for the past 25 years has shone...,1 day ago,Robert Lerman writes that achieving a healthy...,[['https://www.linkedin.com/in/ACoAAACy1HkBviR...,['https://www.urban.org/urban-wire/its-time-mo...,4,0,"[['#workbasedlearning', 'https://www.linkedin....",12,1
1,1,Nicholas Wyman,CEO IWSI Group,6484.0,500+,Nicholas Wyman for the past 25 years has shone...,1 week ago,"National disability advocate Sara Hart Weir, ...",[['https://www.linkedin.com/in/ACoAAAHsfJgBb7_...,[],0,0,[],11,0
2,2,Nicholas Wyman,CEO IWSI Group,6484.0,500+,Nicholas Wyman for the past 25 years has shone...,2 months ago,NaN,[],[],0,0,[],15,0
3,3,Nicholas Wyman,CEO IWSI Group,6484.0,500+,Nicholas Wyman for the past 25 years has shone...,2 months ago,Exploring in this months Talent Management & H...,[['https://www.linkedin.com/in/ACoAAAADlGIBLfn...,['https://www.tlnt.com/apprenticeships-that-br...,4,0,"[['#careerplanning', 'https://www.linkedin.com...",44,0
4,4,Nicholas Wyman,CEO IWSI Group,6484.0,500+,Nicholas Wyman for the past 25 years has shone...,2 months ago,I count myself fortunate to have spent time wi...,[['https://www.linkedin.com/in/ACoAABhNxDUB9IX...,['https://gritdaily.com/the-legacy-of-verified...,3,0,"[['#verifiedresumes', 'https://www.linkedin.co...",22,2
5,5,Nicholas Wyman,CEO IWSI Group,6484.0,500+,Nicholas Wyman for the past 25 years has shone...,2 months ago,Online job platforms are a different way of wo...,[['https://www.linkedin.com/feed/hashtag/?keyw...,['https://www.forbes.com/sites/nicholaswyman/2...,5,0,"[['#careers', 'https://www.linkedin.com/feed/h...",21,1
6,6,Nicholas Wyman,CEO IWSI Group,6484.0,500+,Nicholas Wyman for the past 25 years has shone...,2 months ago,Between the burgeoning unemployment rates and ...,[['https://www.linkedin.com/feed/hashtag/?keyw...,['https://www.theladders.com/career-advice/5-t...,3,0,"[['#hiring', 'https://www.linkedin.com/feed/ha...",16,0
7,7,Nicholas Wyman,CEO IWSI Group,6484.0,500+,Nicholas Wyman for the past 25 years has shone...,3 months ago,This year's National Apprenticeship Week comes...,[['https://www.linkedin.com/in/ACoAABYKFXUBliG...,['https://www.hrdive.com/news/how-are-us-appre...,3,0,"[['#careers', 'https://www.linkedin.com/feed/h...",40,1
8,8,Nicholas Wyman,CEO IWSI Group,6484.0,500+,Nicholas Wyman for the past 25 years has shone...,3 months ago,#coaching #learninganddevelopment #workfor...,[['https://www.linkedin.com/feed/hashtag/?keyw...,['https://www.linkedin.com/pulse/learning-pods...,3,0,"[['#coaching', 'https://www.linkedin.com/feed/...",29,0


In [ ]:
dfc=dfc.dropna()
dfi=dfi.dropna()
print(dfc.shape)
print(dfi.shape)

(11351, 15)
(23951, 15)


In [ ]:
#Shows data type of each column
print(dfc.dtypes)
print(dfi.dtypes)

In [ ]:
#Removes commas from numerical values
dfc['connections'] = dfc['connections'].str.replace(',','')
dfi['connections'] = dfi['connections'].str.replace('+','')
dfc['followers'] = dfc['followers'].str.replace(' followers','')
dfc['followers'] = dfc['followers'].str.replace(',','')

In [ ]:
#changes quantitative data from string to float
dfc['num_hashtags'] = dfc['num_hashtags'].astype(float)
dfi['num_hashtags'] = dfi['num_hashtags'].astype(float)
dfc['reactions'] = dfc['reactions'].astype(float)
dfi['reactions'] = dfi['reactions'].astype(float)
dfc['comments'] = dfc['comments'].astype(float)
dfi['comments'] = dfi['comments'].astype(float)
dfc['connections'] = dfc['connections'].astype(float)
dfi['connections'] = dfi['connections'].astype(float)
dfc['followers'] = dfc['followers'].astype(float)

In [ ]:
#Creates y variable weighting a comment as 2x as important as a reaction
dfc['popularity']=(2*dfc['comments']+dfc['reactions'])/3

In [ ]:
#drops additional unnecessary columns from company data set
dfc = dfc.drop(columns=['about','content','content_links','media_urls','hashtags'])

In [ ]:
print(dfc.shape)

(11351, 11)


In [ ]:
#removes rows of linkedin posts that have been up less that a day
dfc = dfc[~dfc["time_spent"].str.contains('hour')]
dfc = dfc[~dfc["time_spent"].str.contains('hours')]
dfc = dfc[~dfc["time_spent"].str.contains('minutes')]

In [ ]:
print(dfc.shape)

(11258, 11)


In [ ]:
dfc[0:9]

In [ ]:
dfi[0:9]

In [ ]:
import sklearn
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [ ]:
#Create independent variables and dependent variable
dfc_x = dfc[['followers','connections','num_hashtags']]
dfc_y = dfc['popularity']

In [ ]:
#Split the data set into training and testing data (20% testing)
x_train,x_test,y_train,y_test=train_test_split(dfc_x,dfc_y,test_size=0.2)

In [ ]:
dtr=DecisionTreeRegressor()

In [ ]:
dtr.fit(x_train, y_train)

DecisionTreeRegressor()

In [ ]:
score = dtr.score(x_train, y_train)
print("R-squared:", score)

R-squared: 0.22577476470655544


In [ ]:
y_pred = dtr.predict(x_test)
mse = mean_squared_error(y_test, y_pred)
print("MSE: ", mse)

MSE:  203874.616921866


In [ ]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()

# fit the model with the training data
model.fit(x_train,y_train)

# number of trees used
print('Number of Trees used : ', model.n_estimators)


Number of Trees used :  100


In [ ]:
rf_pred = model.predict(x_test)
mserf = mean_squared_error(y_test, rf_pred)
print("MSE: ", mserf)

MSE:  205996.68259815138


In [ ]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()

# fit the model with the training data
lm.fit(x_train,y_train)
predict_test = lm.predict(x_test)


mse_test = mean_squared_error(y_test,predict_test)
scorelm = lm.score(x_train, y_train)
print(mse_test)
print(scorelm)

103468.70309077845
0.012472456696357881


In [ ]:
print(lm.coef_)
print(lm.intercept_)

[ 1.91244052e-05 -2.54105112e-04  1.08777679e+01]
150.09667769081875
